In [ ]:
import torch.nn.functional as F

# 1) 언론사별 통합 편향성 지수 (사전 계산된 값)
outlet_bias_scores = {
    '조선일보':  1.30,
    '동아일보':  0.80,
    '중앙일보':  0.40,
    '한국일보':  0.25,
    'SBS':       0.10,
    '경향신문': -0.90,
    '오마이뉴스': -1.00,
    '한겨레':   -1.20
}

def predict_news_with_probs(text, model, tokenizer, device):
    model.eval()
    enc = tokenizer(text, max_length=512, truncation=True,
                    padding='max_length', return_tensors='pt')
    enc = {k: v.to(device) for k, v in enc.items()}
    with torch.no_grad():
        out = model(**enc)
    # Softmax → 확률
    party_probs     = F.softmax(out['party_logits'],     dim=1).cpu().tolist()[0]
    sentiment_probs = F.softmax(out['sentiment_logits'], dim=1).cpu().tolist()[0]
    # 인덱스→라벨매핑
    party_map     = {0:'국민의힘', 1:'민주당', 2:'그외'}
    sentiment_map = {0:'긍정', 1:'중립', 2:'부정'}
    party_p   = {party_map[i]: p for i, p in enumerate(party_probs)}
    sent_p    = {sentiment_map[i]:p for i, p in enumerate(sentiment_probs)}
    return party_p, sent_p

def compute_final_bias(text, press, model, tokenizer, device, alpha=0.7):
    # 2) 기사별 확률 예측
    party_p, sent_p = predict_news_with_probs(text, model, tokenizer, device)

    # 3) 기사 편향 지수 산정
    # 정당 편향
    article_party_bias = party_p['국민의힘'] - party_p['민주당']
    # 감성 편향에 0.5 가중치
    article_sentiment_bias = (sent_p['긍정'] - sent_p['부정']) / 2
    # 통합 편향성 지수
    article_unified_bias = article_party_bias * (1 + abs(article_sentiment_bias))

    # 4) 언론사 지수 불러오기
    outlet_unified_bias = outlet_bias_scores.get(press, 0.0)

    # 5) 최종 통합 지수 계산
    final_score = alpha * article_unified_bias + (1 - alpha) * outlet_unified_bias

    return {
        'article_party_bias':       article_party_bias,
        'article_sentiment_bias':   article_sentiment_bias,
        'article_unified_bias':     article_unified_bias,
        'outlet_unified_bias':      outlet_unified_bias,
        'final_unified_bias_score': final_score
    }

# --- 사용 예시 ---
new_text  = "정부가 발표한 새로운 정책에 대해 여당은 환영했지만, 야당은 절차적 문제를 지적했다."
new_press = "한겨레"

result = compute_final_bias(new_text, new_press, model, tokenizer, device)
print("최종 통합 편향성 지수:", result['final_unified_bias_score'])
